# Коллоборативная фильтрация. Факторизация матриц

Все началось с конкурса netflix: https://www.netflixprize.com/leaderboard.html

Задача - предсказать какой рейтинг пользователь поставит фильму

Приз - 1 млн долларов

Метрика - RMSE

Задача - улучшить RMSE полученную текущей моделью Netflix на 10%

<img src='netflix_progress.jpg'>

<img src='mf.png'>

Для любого пользователя можно предсказать оценки, которые он еще не ставил. Обучать модель нужно так, чтобы она давала наименьшие ошибки для уже известных моделей.

Предсказание - скалярное произведение вектора для пользователя и вектора для объекта.

При таком подходе необходимо обучить намного меньше параметров модели.

размерность матриц:

- R - n_users x n_items
- P - n_users x n_factors
- Q - n_factors x n_items

n_factors - гиперпараметр, мы сами выбираем

Пример:
- n_users = 1000
- n_items = 2000
- n_factors = 10

- R - 2 000 000 параметров
- P, Q - 30 000 параметров

SVD напрямую применить нельзя, так как очень много отсутствующих значений (точнее можно, как-то заполнив их, но получается не очень хорошее качество).

Поэтому формулируется задача оптимизации по известным оценкам $K$:

### SVD no bias

$r̂_{ui}= q_i^T p_u$

$ min_{p, q} \sum_{u,i \in K} (r_{ui} - q_i^T p_u)^2 + \lambda (||q_i||^2 + ||p_u||^2)$

Как решают эту проблему оптимизации:

1) SGD - берем производные  по $p_u$ и $q_i$

2) ALS - Alternating least squares:
- зафиксируем $p_u$ - задача имеет аналитическое решение для $q_i$ ( метод наименьших квадратов с регурялизацией)
- зафиксируем $q_i$ - задача имеет аналитическое решение для $p_u$

3) Markov Chain Monte Carlo (MCMC) Inference

### SVD biased

$r̂_{ui} = μ + b_u + b_i + q_i^T p_u$

μ - глобальный средний рейтинг

b_u - средний рейтинг для пользователя

b_i - средний рейтинг для item

$ min_{p, q} \sum_{u,i \in K} (r_{ui} - μ + b_u + b_i + q_i^T p_u)^2 + \lambda (b_i^2+b_u^2 + ||q_i||^2 + ||p_u||^2)$

## Пример

данные с оценками фильмов:

https://grouplens.org/datasets/movielens/100k/

https://www.kaggle.com/prajitdatta/movielens-100k-dataset

библиотеки:
- pandas
- surprise
- numpy
- fastFM
- git+git://github.com/scikit-learn/scikit-learn.git


Установить можно так:
```bash
pip install -r requirements.txt
```

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import Reader, Dataset, SVD, evaluate
import numpy as np

## Чтение данных в surprise

In [4]:
# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# или можно загрузить данные из файла / пандас data frame
#data = Dataset.load_from_file('ml-100k/ua.base', reader=Reader(sep='\t'))

In [5]:
# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# # Use the famous SVD algorithm.
svd_bias = SVD()
svd_no_bias = SVD(biased=False)

## Сравним biased и not biased версию SVD

In [6]:
# Run 5-fold cross-validation and print results.
cross_validate(svd_bias, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9331  0.9418  0.9358  0.9329  0.9321  0.9352  0.0036  
MAE (testset)     0.7355  0.7401  0.7388  0.7345  0.7347  0.7367  0.0023  
Fit time          3.34    3.36    3.36    3.33    3.35    3.35    0.01    
Test time         0.12    0.12    0.10    0.12    0.12    0.11    0.01    


{'test_rmse': array([0.93307768, 0.94181229, 0.93583913, 0.93294123, 0.93214   ]),
 'test_mae': array([0.73547964, 0.74013803, 0.73878161, 0.73453653, 0.73467076]),
 'fit_time': (3.3379998207092285,
  3.357976198196411,
  3.3610339164733887,
  3.326995849609375,
  3.354999542236328),
 'test_time': (0.1179955005645752,
  0.11602449417114258,
  0.09599018096923828,
  0.1190023422241211,
  0.11800003051757812)}

In [7]:
# Run 5-fold cross-validation and print results.
cross_validate(svd_no_bias, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9481  0.9467  0.9571  0.9513  0.9506  0.9507  0.0036  
MAE (testset)     0.7462  0.7460  0.7532  0.7499  0.7474  0.7485  0.0027  
Fit time          3.31    3.35    3.34    3.33    3.31    3.33    0.01    
Test time         0.08    0.10    0.08    0.10    0.08    0.09    0.01    


{'test_rmse': array([0.94808664, 0.94672748, 0.95706551, 0.95126171, 0.95058569]),
 'test_mae': array([0.74615834, 0.74601595, 0.75321967, 0.74990738, 0.74735884]),
 'fit_time': (3.313000202178955,
  3.345994472503662,
  3.3420052528381348,
  3.32798171043396,
  3.3090016841888428),
 'test_time': (0.07999563217163086,
  0.10400509834289551,
  0.0789949893951416,
  0.10499954223632812,
  0.07799386978149414)}

# Найдем оптимальные параметры и проверим ошибку на test set

In [8]:
param_grid = {
    'lr_all': [0.005, 0.05],
    'reg_all': [0.02, 0.002],
    'n_factors': [5, 10, 100, 500],
    'n_epochs': [10, 100]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

KeyboardInterrupt: 

In [124]:
print(gs.best_score)

{'rmse': 0.9467960358990256, 'mae': 0.748113763618278}


In [119]:
gs.best_params

{'rmse': {'lr_all': 0.005, 'reg_all': 0.02, 'n_factors': 500, 'n_epochs': 100},
 'mae': {'lr_all': 0.005, 'reg_all': 0.02, 'n_factors': 500, 'n_epochs': 100}}

In [120]:
svd = SVD(**gs.best_params['rmse'])

In [91]:
from surprise.accuracy import mae, rmse

In [148]:
data_train = Dataset.load_from_file('/media/storage/otus/ml-100k/ua.base', reader=Reader(sep='\t'))
data_test = Dataset.load_from_file('/media/storage/otus/ml-100k/ua.test', reader=Reader(sep='\t'))

train_set = data_train.build_full_trainset()
test_set = data_test.build_full_trainset().build_testset()

svd = SVD()

In [150]:
svd.train(train_set)

/media/storage2/anaconda3/envs/otus-dm/lib/python3.6/site-packages/surprise/prediction_algorithms/algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


In [151]:
pred = svd.test(test_set)
mae(pred), rmse(pred)

MAE:  0.7586
RMSE: 0.9606


(0.7585506543927283, 0.9606269779845122)

## то же самое, но через метод predict 

In [161]:
from sklearn.metrics import mean_squared_error, mean_absolute_error


targets, preds = zip(*[(t, svd.predict(u_id, i_id).est) for u_id, i_id, t, _ in data_test.raw_ratings])
print('sklearn MAE: {:.4f}'.format(mean_absolute_error(targets, preds)))
print('sklearn RMSE: {:.4f}'.format(np.sqrt(mean_squared_error(targets, preds))))

sklearn MAE: 0.7586
sklearn RMSE: 0.9606


In [162]:
data_train.raw_ratings[:2]

[('1', '1', 5.0, None), ('1', '2', 3.0, None)]

In [166]:
svd.default_prediction()

3.5238268742409184

In [171]:
print(svd.predict('1', '1'))
print(svd.predict(1, 1))
print('default prediction: {:.2f}'.format(svd.default_prediction()))

user: 1          item: 1          r_ui = None   est = 4.47   {'was_impossible': False}
user: 1          item: 1          r_ui = None   est = 3.52   {'was_impossible': False}
default prediction: 3.52


# Обобщение идеи. Факторизационные машины

Метод SVD рассмотренный выше мы применяли только к матрице user - item.

Что если мы хотим учесть какие-то другие факторы для предсказания рейтинга:
- implicit feedback (учесть страницы фильмов на которые заходил пользователь, но не поставил рейтинг)
- учесть как предпочтения менялись со временем
- учесть дополнительную информацию о продуктах (жанр фильма)
- учесть дополнительную инфо о пользователя (возрастная группа, пол)

Очень много kaggle соревнований на рекомендации были выиграны с помощью факторизационных машин


<img src='fm.png'>


источник: https://www.csie.ntu.edu.tw/~b97053/paper/Factorization%20Machines%20with%20libFM.pdf

$y_i = w_0 + \sum_{j=1:P} w_{ij} x_{ij}  +  \sum_{j=1:P} \sum_{k > j} x_{ij} x_{i_k} < v_j v_k>$

статьи:

https://arxiv.org/pdf/1505.00641.pdf

https://www.csie.ntu.edu.tw/~b97053/paper/Factorization%20Machines%20with%20libFM.pdf

# рассмотрим библиотеку fastFM

http://ibayer.github.io/fastFM/index.html

In [1]:
from fastFM import als, sgd
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import CategoricalEncoder
# pip install git+git://github.com/scikit-learn/scikit-learn.git
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
# информация о фильмах
items_info = pd.read_csv('/media/storage/otus/ml-100k/u.item', header=None, sep='|', encoding="ISO-8859-1")

genres = [
    "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary",
    "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
    "Thriller", "War", "Western"
]

items_info.columns = [
    "item_id", "movie title", "release date", "video release date", "IMDb URL", "unknown"
] + genres
items_info.head()

item_id        movie title release date  video release date  \
0        1   Toy Story (1995)  01-Jan-1995                 NaN   
1        2   GoldenEye (1995)  01-Jan-1995                 NaN   
2        3  Four Rooms (1995)  01-Jan-1995                 NaN   
3        4  Get Shorty (1995)  01-Jan-1995                 NaN   
4        5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0       0   

   Adventure  Animation  Children's   ...     Fantasy  Film-Noir  Horror  \
0          0          1           1   ...           0          0       0   
1          1          0           0   ...           0          0       0   
2          0          0           0   ...           0          0       0   
3          0          0           0   ...           0          0       0   
4          0          0           0   ...           0          0       0   

   Musical  Mystery  Romance  Sci-Fi  Thriller  War  Western  
0        0        0        0       0         0    0        0  
1        0        0        0       0         1    0        0  
2        0        0        0       0         1    0        0  
3        0        0        0       0         0    0        0  
4        0        0        0       0         1    0        0  

[5 rows x 24 columns]

In [221]:
# информация о пользователях

def age_group(x):
    if x < 30:
        return 20
    elif x < 40:
        return 30
    else:
        return 40


users_info = pd.read_csv('/media/storage/otus/ml-100k/u.user', header=None, sep='|')
users_info.columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
users_info['age_gr'] = users_info.age.apply(age_group)
users_info.head()

user_id  age gender  occupation zip_code  age_gr
0        1   24      M  technician    85711      20
1        2   53      F       other    94043      40
2        3   23      M      writer    32067      20
3        4   24      M  technician    43537      20
4        5   33      F       other    15213      30

In [222]:
def read_data(fname):
    data = pd.read_csv(fname, sep='\t', header=None)
    data.columns = ["user_id", "item_id", "rating", "timestamp"]
    return data

def merge_info(df):
    df = df.merge(users_info, on='user_id')
    df = df.merge(items_info, on='item_id')
    return df

In [264]:
df_test.timestamp.median()

883390350.0

In [270]:
(df_train.timestamp - df_test.timestamp.median()) / 60 / 60 / 24 / 12

0       -8.125571
1        4.977354
2        0.201705
3       -5.306205
4       -1.205501
5       -3.795057
6       -5.471178
7       -3.143854
8       -3.590265
9       -8.139570
10      -8.300285
11       2.375641
12       7.677734
13       8.719412
14       7.043316
15      -2.203624
16      -7.151592
17      -4.869825
18      -2.292799
19       4.512250
20      -2.371484
21       8.963057
22       0.297291
23       0.881687
24       4.641882
25      -3.874939
26      -7.371703
27      -3.881304
28      -4.024942
29       0.650971
           ...   
90540   -0.951209
90541    5.152573
90542    5.153321
90543    5.064025
90544    5.699343
90545    7.821914
90546    7.821518
90547    7.821517
90548    6.430336
90549    9.228828
90550    7.821995
90551    7.821995
90552    7.821914
90553    7.821872
90554    7.821952
90555    7.819679
90556    5.391438
90557    7.375938
90558    3.635421
90559    7.372370
90560    7.543723
90561    0.802214
90562   -7.386840
90563   -0.555538
90564    5

In [271]:
df_train = read_data('/media/storage/otus/ml-100k/ua.base')
df_test = read_data('/media/storage/otus/ml-100k/ua.test')

df_train['time'] = (df_train.timestamp - df_train.timestamp.median()) / 60 / 60 / 24 / 12
df_test['time']  = (df_test.timestamp - df_train.timestamp.median()) / 60 / 60 / 24 / 12
df_train.head()

user_id  item_id  rating  timestamp      time
0        1        1       5  874965758 -7.569965
1        1        2       3  876893171 -5.710964
2        1        3       4  878542960 -4.119732
3        1        4       3  876893119 -5.711014
4        1        5       3  889751712  6.691179

In [272]:
df_train = merge_info(df_train)
df_test = merge_info(df_test)
df_train.head()

user_id  item_id  rating  timestamp      time  age gender  occupation  \
0        1        1       5  874965758 -7.569965   24      M  technician   
1        2        1       4  888550871  5.532960   53      F       other   
2        6        1       4  883599478  0.757311   42      M   executive   
3       10        1       4  877888877 -4.750599   53      M      lawyer   
4       13        1       3  882140487 -0.649895   47      M    educator   

  zip_code  age_gr   ...    Fantasy Film-Noir  Horror Musical  Mystery  \
0    85711      20   ...          0         0       0       0        0   
1    94043      40   ...          0         0       0       0        0   
2    98101      40   ...          0         0       0       0        0   
3    90703      40   ...          0         0       0       0        0   
4    29206      40   ...          0         0       0       0        0   

   Romance  Sci-Fi  Thriller  War  Western  
0        0       0         0    0        0  
1        0       0         0    0        0  
2        0       0         0    0        0  
3        0       0         0    0        0  
4        0       0         0    0        0  

[5 rows x 33 columns]

In [225]:
enc = CategoricalEncoder(handle_unknown='ignore')

In [226]:
# fit without users and items info
x_train = enc.fit_transform(df_train[['user_id', 'item_id']]).tocsc()
x_test = enc.transform(df_test[['user_id', 'item_id']]).tocsc()

y_train = df_train['rating']
y_test = df_test['rating']

In [252]:
fm_simple = als.FMRegression(
    rank=5,
    n_iter=100,
    l2_reg=10,
    init_stdev=0.01
)
fm_simple.fit(x_train, y_train)

FMRegression(init_stdev=0.01, l2_reg=10, l2_reg_V=10, l2_reg_w=10, n_iter=100,
       random_state=123, rank=5)

In [253]:
pred_simple = fm_simple.predict(x_test)
print('sklearn MAE: {:.4f}'.format(mean_absolute_error(y_test, pred_simple)))
print('sklearn RMSE: {:.4f}'.format(np.sqrt(mean_squared_error(y_test, pred_simple))))

sklearn MAE: 0.7391
sklearn RMSE: 0.9296


In [256]:
fm_simple2 = sgd.FMRegression(
    rank=10,
    step_size=0.005,
    n_iter=5000000,
    l2_reg=0.02,
    init_stdev=0.01)
fm_simple2.fit(x_train, y_train)

FMRegression(init_stdev=0.01, l2_reg=0.02, l2_reg_V=0.02, l2_reg_w=0.02,
       n_iter=5000000, random_state=123, rank=10, step_size=0.005)

In [257]:
pred_simple2 = fm_simple2.predict(x_test)
print('sklearn MAE: {:.4f}'.format(mean_absolute_error(y_test, pred_simple2)))
print('sklearn RMSE: {:.4f}'.format(np.sqrt(mean_squared_error(y_test, pred_simple2))))

sklearn MAE: 0.7516
sklearn RMSE: 0.9599


In [287]:
print('global mean: {:.2f}'.format(fm_simple.w0_))

global mean: 3.34


In [292]:
fm_simple.w_

array([-0.03617162,  0.08770607, -0.34086575, ..., -0.07635753,
        0.02655206, -0.01308403])

In [294]:
fm_simple.V_[0]

array([ 0.51954781,  0.14751005,  0.18279015, ...,  0.01023122,
       -0.01407122, -0.0069481 ])

# учтем фичи пользователей и фильмов

In [208]:
from scipy import sparse

In [277]:
# fit with users and items info
x_train_attr = enc.fit_transform(df_train[['user_id', 'item_id', 'age_gr', 'gender']]).tocsc()
x_test_attr = enc.transform(df_test[['user_id', 'item_id', 'age_gr', 'gender']]).tocsc()

# items information
items_train = sparse.csc_matrix(df_train[genres].values)
items_test = sparse.csc_matrix(df_test[genres].values)

# time information
time_train = sparse.csc_matrix(df_train['time'].values.reshape(-1, 1))
time_test = sparse.csc_matrix(df_test['time'].values.reshape(-1, 1))


x_train_attr = sparse.hstack([x_train_attr, items_train])
x_test_attr = sparse.hstack([x_test_attr, items_test])

x_train_attr_time = sparse.hstack([x_train_attr, items_train, time_train])
x_test_attr_time = sparse.hstack([x_test_attr, items_test, time_test])

print(x_train_attr.shape, x_test_attr.shape)

(90570, 2646) (9430, 2646)


In [278]:
fm = als.FMRegression(
    rank=5,
    n_iter=100,
    l2_reg=10,
    init_stdev=0.01
)

fm.fit(x_train_attr, y_train)

FMRegression(init_stdev=0.01, l2_reg=10, l2_reg_V=10, l2_reg_w=10, n_iter=100,
       random_state=123, rank=5)

In [279]:
pred_attr = fm.predict(x_test_attr)
print('sklearn MAE: {:.4f}'.format(mean_absolute_error(y_test, pred_attr)))
print('sklearn RMSE: {:.4f}'.format(np.sqrt(mean_squared_error(y_test, pred_attr))))

sklearn MAE: 0.7320
sklearn RMSE: 0.9283


In [281]:
fm2 = als.FMRegression(
    rank=5,
    n_iter=100,
    l2_reg=10,
    init_stdev=0.01
)

fm2.fit(x_train_attr_time, y_train)

FMRegression(init_stdev=0.01, l2_reg=10, l2_reg_V=10, l2_reg_w=10, n_iter=100,
       random_state=123, rank=5)

In [283]:
pred_attr = fm2.predict(x_test_attr_time)
print('sklearn MAE: {:.4f}'.format(mean_absolute_error(y_test, pred_attr)))
print('sklearn RMSE: {:.4f}'.format(np.sqrt(mean_squared_error(y_test, pred_attr))))

sklearn MAE: 0.7291
sklearn RMSE: 0.9226


## Эффект различных слагаемых на качество предсказания рейтинга



<img src='effect_factorizations.png'>

источник:

https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf

# Оценка качества рекомендательной системы

* Качество рейтингов
    * MAE, MSE
* Качество событий
    * F-score, ROC-AUC, PR-AUC
* Качество ранжирования
    * precision@k, recall@k
    * Mean average precision@k
    * $DCG@k(u) = \sum\limits_{p=1}^k \frac{val(i,p)}{\log{(p+1)}}$
    * $nDCG@k(u) = \frac{DCG@k(u)}{\max{(DCG@k(u))}}$


ссылки:

https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Offline_metrics

https://habr.com/company/econtenta/blog/303458/

# Ассоциативные правила

Рекомендации вида: «Кто купил x, также купил y».

Дано: база транзакций / покупок

Хотим посчитать: 
- P(X ...) - support(X ...)
- P(Y|X) - сonfidence(X->Y)
- Lift(X->Y) = support(X and Y) / (support(X) * support(Y))
- Conviction(X->Y) = (1 - support(Y)) / (1 - confidence(X->Y))

где X, Y - товары


Эффективный рассчет величин выше: Apriori Algorithm


Статья:

https://habr.com/company/ods/blog/353502/

Полезные ссылки:
1. https://habrahabr.ru/company/surfingbird/blog/139518/
2. https://d4datascience.wordpress.com/category/predictive-analytics/
3. Data Science from scratch
4. https://habrahabr.ru/company/yandex/blog/241455/
5. https://www.kaggle.com/rounakbanik/movie-recommender-systems/notebook
6. http://www.cs.ubbcluj.ro/~gabis/DocDiplome/SistemeDeRecomandare/Recommender_systems_handbook.pdf
7. https://www.coursera.org/specializations/recommender-systems
8. https://www.cs.umd.edu/~samir/498/Amazon-Recommendations.pdf
9. https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf
10. http://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD
11. https://arxiv.org/pdf/1505.00641.pdf
12. https://www.csie.ntu.edu.tw/~b97053/paper/Factorization%20Machines%20with%20libFM.pdf
13. https://habr.com/company/ods/blog/353502/

## Опрос в конце лекции:
https://docs.google.com/forms/d/e/1FAIpQLSccnSyGVvKKcYvuujeKVp9hchm8sea6pCl2IpdoNstM8-Y7vg/viewform?usp=sf_link